In this homework, we'll deploy the Bank Marketing model from the homework 5. We already have a docker image for this model - we'll use it for deploying the model to Kubernetes. 

### Question 1: Deploy the model to Kubernetes

The command to run the docker container is:
``` bash
docker pull svizor/zoomcamp-model:3.11.5-hw10
docker run -it -p 9696:9696 svizor/zoomcamp-model:3.11.5-hw10
```


To test the model we can use the following code:
``` python 
python q6_test.py


This is the output of the test:
```
python q6_test.py
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
```

### Question 2

In [1]:
!powershell kind version

kind v0.25.0 go1.22.9 windows/amd64


In [2]:
!powershell kind create cluster # create a cluster with the default name

Creating cluster "kind" ...
 • Ensuring node image (kindest/node:v1.31.2) 🖼  ...
 ✓ Ensuring node image (kindest/node:v1.31.2) 🖼
 • Preparing nodes 📦   ...
 ✓ Preparing nodes 📦 
 • Writing configuration 📜  ...
 ✓ Writing configuration 📜
 • Starting control-plane 🕹️  ...
 ✓ Starting control-plane 🕹️
 • Installing CNI 🔌  ...
 ✓ Installing CNI 🔌
 • Installing StorageClass 💾  ...
 ✓ Installing StorageClass 💾
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Not sure what to do next? 😅  Check out https://kind.sigs.k8s.io/docs/user/quick-start/


In [3]:
!powershell kubectl cluster-info # check if the cluster is running 

Kubernetes control plane is running at https://127.0.0.1:52651
CoreDNS is running at https://127.0.0.1:52651/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


### Question 3

The smallest deployment computing unit in kubernetes is a pod. A pod is a group of one or more containers, with shared storage/network resources, and a specification for how to run the containers.

### Question 4

In [5]:
!powershell kubectl get service # check the services running in the cluster

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   4m46s


### Question 5

In [10]:
!powershell kind load docker-image zoomcamp-model:3.11.5-hw10 # load the docker image into the cluster

Image: "zoomcamp-model:3.11.5-hw10" with ID "sha256:020904a2523cec81c854a9b4679ae26d23f814e42350721136d1447f910a6b53" found to be already present on all nodes.


### Question 6

image: zoomcamp-model:3.11.5-hw10 # the image name \
memory: "128Mi" # 128 mebibytes of memory\
cpu: "200m" # 0.2 of a core\
port : 9696 # the port the model is running on

In [21]:
!powershell kubectl apply -f deployment.yaml # apply the deployment.yaml file to the cluster

deployment.apps/subscription configured


In [22]:
!powershell kubectl get pods # check the pods running in the cluster

NAME                            READY   STATUS              RESTARTS   AGE
subscription-544b4f9664-jr8zd   1/1     Running             0          24m
subscription-7b74b5846-p47hz    0/1     ContainerCreating   0          2s


### Question 7

In [23]:
!powershell kubectl apply -f service.yaml # apply the service.yaml file to the cluster

service/subscription unchanged


In [29]:
!powershell kubectl get services # check the services running in the cluster 

NAME           TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
kubernetes     ClusterIP      10.96.0.1      <none>        443/TCP        83m
subscription   LoadBalancer   10.96.30.244   <pending>     80:30091/TCP   56m


In [32]:
!powershell kubectl get pods # check the pods running in the cluster

NAME                           READY   STATUS    RESTARTS   AGE
subscription-7b74b5846-p47hz   1/1     Running   0          39m


In [38]:
!powershell kubectl port-forward pod/subscription-7b74b5846-p47hz 9696:9696 # port forward the pod to the localhost

^C


Forwarding from 127.0.0.1:9696 -> 9696
Forwarding from [::1]:9696 -> 9696


In [39]:
!python q6_test.py # run the test script to check if the service is working correctly

{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}


This is the same result as the one we got above when we ran just the docker container. 

### Followup HPA

In [40]:
!powershell kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3 # autoscale the deployment

horizontalpodautoscaler.autoscaling/subscription-hpa autoscaled


In [41]:
!powershell kubectl get hpa # check the horizontal pod autoscaler

NAME               REFERENCE                 TARGETS              MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: <unknown>/20%   1         3         0          10s


In [44]:
!powershell kubectl get pods -n kube-system # check the pods running in the kube-system namespace


NAME                                         READY   STATUS    RESTARTS   AGE
coredns-7c65d6cfc9-6zrx8                     1/1     Running   0          110m
coredns-7c65d6cfc9-s94tr                     1/1     Running   0          110m
etcd-kind-control-plane                      1/1     Running   0          110m
kindnet-pjxks                                1/1     Running   0          110m
kube-apiserver-kind-control-plane            1/1     Running   0          110m
kube-controller-manager-kind-control-plane   1/1     Running   0          110m
kube-proxy-xffh8                             1/1     Running   0          110m
kube-scheduler-kind-control-plane            1/1     Running   0          110m


In [45]:
!powershell kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml # apply the metrics server

serviceaccount/metrics-server unchanged
clusterrole.rbac.authorization.k8s.io/system:aggregated-metrics-reader unchanged
clusterrole.rbac.authorization.k8s.io/system:metrics-server unchanged
rolebinding.rbac.authorization.k8s.io/metrics-server-auth-reader unchanged
clusterrolebinding.rbac.authorization.k8s.io/metrics-server:system:auth-delegator unchanged
clusterrolebinding.rbac.authorization.k8s.io/system:metrics-server unchanged
service/metrics-server unchanged
deployment.apps/metrics-server configured
apiservice.apiregistration.k8s.io/v1beta1.metrics.k8s.io unchanged


In [51]:
!powershell kubectl get hpa # check the horizontal pod autoscaler

NAME               REFERENCE                 TARGETS              MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: <unknown>/20%   1         3         1          6m55s


In [53]:
!powershell kubectl get pods -n kube-system # check the pods running in the kube-system namespace

NAME                                         READY   STATUS    RESTARTS   AGE
coredns-7c65d6cfc9-6zrx8                     1/1     Running   0          123m
coredns-7c65d6cfc9-s94tr                     1/1     Running   0          123m
etcd-kind-control-plane                      1/1     Running   0          123m
kindnet-pjxks                                1/1     Running   0          123m
kube-apiserver-kind-control-plane            1/1     Running   0          123m
kube-controller-manager-kind-control-plane   1/1     Running   0          123m
kube-proxy-xffh8                             1/1     Running   0          123m
kube-scheduler-kind-control-plane            1/1     Running   0          123m
metrics-server-54bf7cdd6-9gw2v               0/1     Running   0          12m


Unfortunately, I was not able to complete the followup HPA. I was able to create the HPA, but I was not able to test it. I was getting an error that the HPA was not able to get the metrics from the pod. I tried to fix it, but I was not able to. 

Will get back to this later. 